In [0]:
!tar zxvf ./drive/My\ Drive/Colab\ Notebooks/train.tgz
!pip install wandb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: f8e6571f72d1337d268b4c1bba4daa491fb9e6fd
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# CAPTCHA Recognition - 6 digits

In [4]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [5]:
from tensorflow.keras import backend
from tensorflow.python.keras import layers

In [3]:
import wandb

# Parsing Data

In [6]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    return img

In [7]:
#training data 01
def prepData01():  
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_data = train_dir.map(lambda x: readimg_to_tensor(x))
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01=tf.data.Dataset.zip((train_data,train_label))
  validation_split=0.1
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(50000,seed = 1).batch(50000))

  img , lb = next(train_data_gen)
  lb = tf.concat([tf.expand_dims(lbs,1) for lbs in lb],1)
  img_train = img[:-split]
  lb_train = lb[:-split]
  img_test = img[-split:]
  lb_test = lb[-split:]
  return img_train, lb_train, img_test, lb_test

In [8]:
#training data 02
def prepData02(): 
  train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  data2 = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data2)):
      data2.iloc[i,1] = list(data2.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data2)):
          arr[j][i] = data2.iloc[i,1][j]
  data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  #to one hot
  for i in range(6):
      data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

  train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
  train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
  train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data02=tf.data.Dataset.zip((train_data2,train_label2))
  validation_split=0.1
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data02.shuffle(50000,seed = 1).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [9]:
#number of element
#num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements=50000

# Model configeration

In [6]:
config = {
    "stack" : "16+ 16,32,64,128 each 2 blocks",
    "weight_regu" : 0.0001,
    "dropout" : "0.3 at each blocks",
    "preact" : True,
    "name" : "ResNet_small_4_mixSGD",
    "output_WR" : 1,
    "total_params" : 2211352
}

In [52]:
wandb.init(project="my-project-2",reinit=True,name=config["name"], config=config)

W&B Run: https://app.wandb.ai/tingww/my-project-2/runs/23dz5kwm

In [10]:
wandb.init(id='23dz5kwm',resume='allow')

W&B Run: https://app.wandb.ai/tingww/uncategorized/runs/23dz5kwm

In [0]:
id = wandb.util.generate_id()

In [54]:
id

'9893o1xm'

In [9]:
#ini and regul
initializer = tf.keras.initializers.he_normal( seed = 3)
alpha = 1e-5  # weight decay coefficient
regularizer = tf.keras.regularizers.l2(alpha)

In [7]:
def block2(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
  """A residual block.
  Arguments:
      x: input tensor.
      filters: integer, filters of the bottleneck layer.
      kernel_size: default 3, kernel size of the bottleneck layer.
      stride: default 1, stride of the first layer.
      conv_shortcut: default False, use convolution shortcut if True,
        otherwise identity shortcut.
      name: string, block label.
  Returns:
    Output tensor for the residual block.
  """
  bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

  preact = layers.BatchNormalization(epsilon=1.001e-5, name=name + '_preact_bn')(x)
  preact = layers.Activation('relu', name=name + '_preact_relu')(preact)

  if conv_shortcut:
    shortcut = layers.Conv2D(
        4 * filters, 1, strides=stride, name=name + '_0_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  else:
    shortcut = layers.MaxPooling2D(1, strides=stride)(x) if stride > 1 else x

  x = layers.Conv2D(
      filters, 1, strides=1, use_bias=False, name=name + '_1_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(preact)
  x = layers.BatchNormalization(epsilon=1.001e-5, name=name + '_1_bn')(x)
  x = layers.Activation('relu', name=name + '_1_relu')(x)

  x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name + '_2_pad')(x)
  x = layers.Conv2D(
      filters,
      kernel_size,
      strides=stride,
      use_bias=False,
      name=name + '_2_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  x = layers.BatchNormalization(epsilon=1.001e-5, name=name + '_2_bn')(x)
  x = layers.Activation('relu', name=name + '_2_relu')(x)

  x = layers.Conv2D(4 * filters, 1, name=name + '_3_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Add(name=name + '_out')([shortcut, x])
  return x

def stack1(x, filters, blocks, stride1=2, name=None):
  """A set of stacked residual blocks.
  Arguments:
    x: input tensor.
    filters: integer, filters of the bottleneck layer in a block.
    blocks: integer, blocks in the stacked blocks.
    stride1: default 2, stride of the first layer in the first block.
    name: string, stack label.
  Returns:
    Output tensor for the stacked blocks.
  """
  x = block2(x, filters, conv_shortcut=True ,stride=stride1, name=name + '_block1')
  for i in range(2, blocks + 1):
    x = block2(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
  return x

In [12]:
def build_ResNet_Model(modelname="myResNet"):

  def stack_fn(x):
    x = stack1(x, 32, 2, stride1=1, name='conv2')
    x = stack1(x, 64, 2, name='conv3')
    x = stack1(x, 128, 2, name='conv4')
    return stack1(x, 256, 2, name='conv5')
  #input shape
  img_input = layers.Input(shape=(60,200,3))

  x = layers.ZeroPadding2D(
      padding=((2, 2)), name='conv1_pad')(img_input)
  x = layers.Conv2D(64, 3, strides=1, name='conv1_conv',kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  #pre act
  x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name='pool1_pad')(x)
  x = layers.MaxPooling2D(3, strides=2, name='pool1_pool')(x)
  x = layers.Dropout(0.3)(x)
  x = stack_fn(x)
  
  x = layers.BatchNormalization(epsilon=1.001e-5, name='post_bn')(x)
  x = layers.Activation('relu', name='post_relu')(x)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)
  x = layers.Flatten()(x)

  #classified
  digit1 = layers.Dense(36, name='digit1', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit2 = layers.Dense(36, name='digit2', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit3 = layers.Dense(36, name='digit3', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit4 = layers.Dense(36, name='digit4', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit5 = layers.Dense(36, name='digit5', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit6 = layers.Dense(36, name='digit6', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  modelt = tf.keras.models.Model(inputs=img_input,outputs = [digit1 , digit2, digit3, digit4, digit5, digit6], name = modelname)
  

  

  modelt.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), 
    metrics=['accuracy'])
  return modelt

In [18]:
def trainModel(model,img_train,lb_train,callB):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = 64,
        shuffle = True,
        validation_split=0.1,
        epochs=100,
        verbose=1,
        callbacks = callB
        )

In [0]:
filepath = "/content/drive/My Drive/Colab Notebooks/Saved Model/"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath+config["name"]+".hdf5", monitor='val_loss', verbose=1, save_best_only=True,mode='min')

In [0]:
img , lb= prepData01()

In [13]:
mymodel = build_ResNet_Model()

In [14]:
mymodel.summary()

Model: "myResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 64, 204, 3)   0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 62, 202, 64)  1792        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 64, 204, 64)  0           conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
mymodel.load_weights(filepath+"ResNet_small_4.hdf5")

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [22]:
mymodel.optimizer

In [0]:
mymodel.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), 
    metrics=['accuracy'])

In [0]:
callbacks_list = [checkpoint , wandb.keras.WandbCallback()]
trainModel(mymodel,img,lb,callbacks_list)

Epoch 1/100
703/704 [============================>.] - ETA: 0s - loss: 0.6134 - digit1_loss: 0.0148 - digit2_loss: 0.0123 - digit3_loss: 0.0101 - digit4_loss: 0.0108 - digit5_loss: 0.0159 - digit6_loss: 0.0134 - digit1_accuracy: 0.9955 - digit2_accuracy: 0.9960 - digit3_accuracy: 0.9970 - digit4_accuracy: 0.9968 - digit5_accuracy: 0.9953 - digit6_accuracy: 0.9961
Epoch 00001: val_loss did not improve from 0.61597
704/704 [==============================] - 44s 63ms/step - loss: 0.6134 - digit1_loss: 0.0148 - digit2_loss: 0.0123 - digit3_loss: 0.0101 - digit4_loss: 0.0108 - digit5_loss: 0.0159 - digit6_loss: 0.0134 - digit1_accuracy: 0.9955 - digit2_accuracy: 0.9960 - digit3_accuracy: 0.9970 - digit4_accuracy: 0.9968 - digit5_accuracy: 0.9953 - digit6_accuracy: 0.9961 - val_loss: 0.7431 - val_digit1_loss: 0.0316 - val_digit2_loss: 0.0292 - val_digit3_loss: 0.0301 - val_digit4_loss: 0.0372 - val_digit5_loss: 0.0454 - val_digit6_loss: 0.0343 - val_digit1_accuracy: 0.9908 - val_digit2_accur